In [ ]:
!pip install geopandas ipyleaflet

In [1]:
import ipyleaflet as ipy 
import ipywidgets as ipyw
import geopandas 
import json

In [2]:
from ipyleaflet import Map, Marker

# find features for a coordinate

x_coord = -34.81029365869138
y_coord = 148.98718863300078

map = ipy.Map(center=[x_coord, y_coord], zoom=5)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

coord = (x_coord, y_coord)

marker = Marker(location=coord, draggable=False)
map.add_layer(marker);
ipyw.VBox([map, label])

In [3]:
#query loci api - find by location for a input dataset feature type

featureType = "asgs16_sa4"

import urllib.request
url = 'https://api2.loci.cat/api/v1/location/find_at_location?loci_type={}&lat={}&lon={}&count=1000&offset=0'.format(featureType, x_coord, y_coord)
print(url)
res = urllib.request.urlopen(url)
res_body = res.read()
j = json.loads(res_body.decode("utf-8"))
item = j['locations']['res'][0]
print(item)

https://api2.loci.cat/api/v1/location/find_at_location?loci_type=asgs16_sa4&lat=-34.81029365869138&lon=148.98718863300078&count=1000&offset=0
{'id': '101', 'dataset': 'asgs16_sa4', 'geometry': 'https://gds.loci.cat/geometry/asgs16_sa4/101', 'feature': 'http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel4/101'}


In [4]:
feature_uri  = item['feature']
feature_uri

'http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel4/101'

In [5]:
#get info about the feature (RDF)
header={'Accept': 'text/turtle'}
req = urllib.request.Request(url=feature_uri, headers=header, method='GET')
res = urllib.request.urlopen(req)
res_body = res.read().decode("utf-8")
print(res_body)

@prefix asgs: <http://linked.data.gov.au/def/asgs#> .
@prefix data: <http://linked.data.gov.au/def/datatype/> .
@prefix geo: <http://www.opengis.net/ont/geosparql#> .
@prefix geox: <http://linked.data.gov.au/def/geox#> .
@prefix ns1: <http://purl.org/linked-data/registry#> .
@prefix ns2: <http://www.w3.org/ns/qb4st/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://linked.data.gov.au/dataset/asgs2016/stateorterritory/1> asgs:isStateOrTerritoryOf <http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel4/101> .

<http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel4/101> a asgs:StatisticalAreaLevel4,
        geo:Feature ;
    asgs:sa4Code2016 "101" ;
    asgs:sa4Name2016 "Capital Region" ;
    asgs:statisticalArea4Sa43DigitCode "101" ;
    geox:hasAreaM2 [ data:value 51895563100.0 ;
     

In [6]:
geom_uri = item['geometry']
geom_uri

'https://gds.loci.cat/geometry/asgs16_sa4/101'

In [7]:
# get semantic info about the geom
header={'Accept': 'text/turtle'}
url = geom_uri + "?_view=simplifiedgeom"
req = urllib.request.Request(url=geom_uri, headers=header, method='GET')
res = urllib.request.urlopen(req)
res_body = res.read().decode("utf-8")

In [8]:
# get the geojson representation of the geom
header={'Accept': 'application/json'}
req = urllib.request.Request(url=geom_uri, headers=header, method='GET')
res = urllib.request.urlopen(req)
res_body = res.read().decode("utf-8")
geojson_data = json.loads(res_body)

In [9]:
#draw the geometry for the feature on a map

from ipyleaflet import GeoJSON

geo_json = GeoJSON(data=geojson_data, style = {'color': 'green', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.1})
map.add_layer(geo_json)
map


Map(center=[-34.81029365869138, 148.98718863300078], controls=(ZoomControl(options=['position', 'zoom_in_text'…